# Анализ источников новостей

Цель данного исследования: проанализировать источники новостных лент, сравнить рубрики, подумать над возможностью добавление\удаление категорий для проекта "Радар новостей рунета"

In [1]:
import numpy as np
import pandas as pd
import datetime as DT

## Meduza

Загрузим датасет meduza. Одновременно сделаем преобразование даты из строки

In [2]:
dateparse = lambda x: pd.datetime.strptime(x, "%Y-%m-%d %H:%M:%S")

In [5]:
data_meduza = pd.read_csv('../data/meduza.csv',
                      sep=',',
                      index_col=False,
                      escapechar='\\',
                      parse_dates=['date'],
                      date_parser=dateparse,
                      quotechar='\"',
                      error_bad_lines=False)

b'Skipping line 47579: expected 17 fields, saw 19\n'


In [6]:
# Проверим преобразование
data_meduza.dtypes

date           datetime64[ns]
url                    object
edition                object
topics                float64
authors               float64
title                  object
text                   object
reposts_fb             object
reposts_vk             object
reposts_ok             object
reposts_twi            object
reposts_lj             object
reposts_tg             object
likes                  object
views                  object
comm_count             object
dtype: object

In [7]:
data_meduza.shape

(71806, 16)

In [9]:
# Так выглядет датасет Meduza
data_meduza.head()

,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
0,2019-08-06 14:15:00,https://meduza.io/feature/2019/08/06/matros-ra...,-,NaN,NaN,Матрос разговаривал в казарме по видеосвязи,С марта 2019 года российским военным запрещено...,-,-,-,-,-,-,-,-,-
1,2019-08-06 14:20:00,https://meduza.io/news/2019/08/06/umerla-feshn...,-,NaN,NaN,Умерла фэшн-директор ЦУМа Алла Вербер,"Умерла фэшн-директор московского ЦУМа, вице-пр...",-,-,-,-,-,-,-,-,-
2,2019-08-06 14:10:00,https://meduza.io/feature/2019/08/06/sama-eta-...,-,NaN,NaN,Сама эта мысль казалась дурацкой: роботы-пчелы...,13 августа в издательстве «Эксмо» выходит книг...,-,-,-,-,-,-,-,-,-
3,2019-08-06 14:35:00,https://meduza.io/news/2019/08/06/u-zaderzhann...,-,NaN,NaN,У задержанных на акции за свободные выборы 3 а...,"Некоторые участникиnакции 3 августаn, у которы...",-,-,-,-,-,-,-,-,-
4,2019-08-06 15:00:00,https://meduza.io/feature/2019/08/06/pochemu-v...,-,NaN,NaN,Почему ваш товар плохо продается в интернете?,"Интернет-покупатели становятся все капризнее, ...",-,-,-,-,-,-,-,-,-


In [10]:
# Выясним в каким столбцах есть пустые данные
data_meduza.isnull().any()

date           False
url            False
edition        False
topics          True
authors         True
title           True
text            True
reposts_fb     False
reposts_vk     False
reposts_ok     False
reposts_twi    False
reposts_lj     False
reposts_tg     False
likes          False
views          False
comm_count     False
dtype: bool

Как видим, в некоторых столбцах вместо значения Nan используется "-", их преобразованием займемся позже. А пока посмотрим строки, в которых пустые тексты новостей

In [11]:
print('Кол-во строк с значением NaN в столбце "topics" = ', data_meduza['topics'].isnull().sum())
data_meduza[data_meduza['topics'].isnull()]

Кол-во строк с значением NaN в столбце "topics" =  71806


,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
0,2019-08-06 14:15:00,https://meduza.io/feature/2019/08/06/matros-ra...,-,NaN,NaN,Матрос разговаривал в казарме по видеосвязи,С марта 2019 года российским военным запрещено...,-,-,-,-,-,-,-,-,-
1,2019-08-06 14:20:00,https://meduza.io/news/2019/08/06/umerla-feshn...,-,NaN,NaN,Умерла фэшн-директор ЦУМа Алла Вербер,"Умерла фэшн-директор московского ЦУМа, вице-пр...",-,-,-,-,-,-,-,-,-
2,2019-08-06 14:10:00,https://meduza.io/feature/2019/08/06/sama-eta-...,-,NaN,NaN,Сама эта мысль казалась дурацкой: роботы-пчелы...,13 августа в издательстве «Эксмо» выходит книг...,-,-,-,-,-,-,-,-,-
3,2019-08-06 14:35:00,https://meduza.io/news/2019/08/06/u-zaderzhann...,-,NaN,NaN,У задержанных на акции за свободные выборы 3 а...,"Некоторые участникиnакции 3 августаn, у которы...",-,-,-,-,-,-,-,-,-
4,2019-08-06 15:00:00,https://meduza.io/feature/2019/08/06/pochemu-v...,-,NaN,NaN,Почему ваш товар плохо продается в интернете?,"Интернет-покупатели становятся все капризнее, ...",-,-,-,-,-,-,-,-,-
5,2019-08-06 15:14:00,https://meduza.io/news/2019/08/06/kandidata-v-...,-,NaN,NaN,Кандидата в Мосгордуму Юлию Галямину арестовал...,Чертановский суд Москвы арестовал незарегистри...,-,-,-,-,-,-,-,-,-
6,2019-08-06 15:19:00,https://meduza.io/feature/2019/08/06/zhalko-ch...,-,NaN,NaN,"«Жалко, что большинство участников митинга — н...",Проректор Высшей школы экономики Валерия Касам...,-,-,-,-,-,-,-,-,-
7,2019-08-06 15:20:00,https://meduza.io/feature/2019/08/06/more-ognya,-,NaN,NaN,Море огня,Вертолет тушит лесной пожар в Красноярском кра...,-,-,-,-,-,-,-,-,-
8,2019-08-06 15:43:00,https://meduza.io/short/2019/08/06/skolko-pres...,-,NaN,NaN,Сколько преступлений выявили в силовых структу...,Газета «Новые известия»nполучилаnот Генпрокура...,-,-,-,-,-,-,-,-,-
9,2019-08-06 15:43:00,https://meduza.io/news/2019/08/06/posol-ssha-v...,-,NaN,NaN,Посол США в России Джон Хантсман подал в отставку,Посол США в РФ Джон Хантсман подал президенту ...,-,-,-,-,-,-,-,-,-


In [14]:
print('Кол-во строк с значением NaN в столбце "Text" = ', data_meduza['text'].isnull().sum())
data_meduza[data_meduza['text'].isnull()]

Кол-во строк с значением NaN в столбце "Text" =  1372


,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
33,2019-08-06 10:53:00,https://meduza.io/quiz/otlichite-kachestvennyy...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
86,2019-08-05 14:41:00,https://meduza.io/quiz/moscow-is-the-capital-o...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
173,2019-08-02 13:05:00,https://meduza.io/quiz/kostyl-ili-chekvarka,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
194,2019-08-02 05:30:00,https://meduza.io/quiz/chto-obsuzhdayut-na-ros...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
250,2019-08-01 08:39:00,https://meduza.io/games/v-rossii-mogut-oshtraf...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
273,2019-07-31 15:09:00,https://meduza.io/survey/skolko-ostavlyat-chae...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
326,2019-07-30 16:18:00,https://meduza.io/quiz/rossiya-ili-zimbabve-pr...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
339,2019-07-30 08:08:00,https://meduza.io/games/eto-skazal-patsan-ili-...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
361,2019-07-29 14:45:00,https://meduza.io/quiz/kto-vvel-nalog-na-borod...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
385,2019-07-29 07:00:00,https://meduza.io/quiz/eto-lisichka-ili-pogank...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-


In [15]:
print('Кол-во строк с значением NaN в столбце "authors" = ', data_meduza['authors'].isnull().sum())
data_meduza[data_meduza['authors'].isnull()]

Кол-во строк с значением NaN в столбце "authors" =  71806


,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
0,2019-08-06 14:15:00,https://meduza.io/feature/2019/08/06/matros-ra...,-,NaN,NaN,Матрос разговаривал в казарме по видеосвязи,С марта 2019 года российским военным запрещено...,-,-,-,-,-,-,-,-,-
1,2019-08-06 14:20:00,https://meduza.io/news/2019/08/06/umerla-feshn...,-,NaN,NaN,Умерла фэшн-директор ЦУМа Алла Вербер,"Умерла фэшн-директор московского ЦУМа, вице-пр...",-,-,-,-,-,-,-,-,-
2,2019-08-06 14:10:00,https://meduza.io/feature/2019/08/06/sama-eta-...,-,NaN,NaN,Сама эта мысль казалась дурацкой: роботы-пчелы...,13 августа в издательстве «Эксмо» выходит книг...,-,-,-,-,-,-,-,-,-
3,2019-08-06 14:35:00,https://meduza.io/news/2019/08/06/u-zaderzhann...,-,NaN,NaN,У задержанных на акции за свободные выборы 3 а...,"Некоторые участникиnакции 3 августаn, у которы...",-,-,-,-,-,-,-,-,-
4,2019-08-06 15:00:00,https://meduza.io/feature/2019/08/06/pochemu-v...,-,NaN,NaN,Почему ваш товар плохо продается в интернете?,"Интернет-покупатели становятся все капризнее, ...",-,-,-,-,-,-,-,-,-
5,2019-08-06 15:14:00,https://meduza.io/news/2019/08/06/kandidata-v-...,-,NaN,NaN,Кандидата в Мосгордуму Юлию Галямину арестовал...,Чертановский суд Москвы арестовал незарегистри...,-,-,-,-,-,-,-,-,-
6,2019-08-06 15:19:00,https://meduza.io/feature/2019/08/06/zhalko-ch...,-,NaN,NaN,"«Жалко, что большинство участников митинга — н...",Проректор Высшей школы экономики Валерия Касам...,-,-,-,-,-,-,-,-,-
7,2019-08-06 15:20:00,https://meduza.io/feature/2019/08/06/more-ognya,-,NaN,NaN,Море огня,Вертолет тушит лесной пожар в Красноярском кра...,-,-,-,-,-,-,-,-,-
8,2019-08-06 15:43:00,https://meduza.io/short/2019/08/06/skolko-pres...,-,NaN,NaN,Сколько преступлений выявили в силовых структу...,Газета «Новые известия»nполучилаnот Генпрокура...,-,-,-,-,-,-,-,-,-
9,2019-08-06 15:43:00,https://meduza.io/news/2019/08/06/posol-ssha-v...,-,NaN,NaN,Посол США в России Джон Хантсман подал в отставку,Посол США в РФ Джон Хантсман подал президенту ...,-,-,-,-,-,-,-,-,-


In [16]:
print('Кол-во строк с значением NaN в столбце "title" = ', data_meduza['title'].isnull().sum())
data_meduza[data_meduza['title'].isnull()]

Кол-во строк с значением NaN в столбце "title" =  1244


,date,url,edition,topics,authors,title,text,reposts_fb,reposts_vk,reposts_ok,reposts_twi,reposts_lj,reposts_tg,likes,views,comm_count
33,2019-08-06 10:53:00,https://meduza.io/quiz/otlichite-kachestvennyy...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
86,2019-08-05 14:41:00,https://meduza.io/quiz/moscow-is-the-capital-o...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
98,2019-08-05 05:00:00,https://meduza.io/feature/2019/08/05/v-poiskah...,-,NaN,NaN,NaN,В начале января 2006 года в галерее Сюзанны Ца...,-,-,-,-,-,-,-,-,-
173,2019-08-02 13:05:00,https://meduza.io/quiz/kostyl-ili-chekvarka,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
194,2019-08-02 05:30:00,https://meduza.io/quiz/chto-obsuzhdayut-na-ros...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
250,2019-08-01 08:39:00,https://meduza.io/games/v-rossii-mogut-oshtraf...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
273,2019-07-31 15:09:00,https://meduza.io/survey/skolko-ostavlyat-chae...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
326,2019-07-30 16:18:00,https://meduza.io/quiz/rossiya-ili-zimbabve-pr...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
339,2019-07-30 08:08:00,https://meduza.io/games/eto-skazal-patsan-ili-...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-
361,2019-07-29 14:45:00,https://meduza.io/quiz/kto-vvel-nalog-na-borod...,-,NaN,NaN,NaN,NaN,-,-,-,-,-,-,-,-,-


In [157]:
#Удалим эти строки
data_meduza['text'].dropna(inplace=True)

In [158]:
print('Кол-во строк с значением NaN в столбце "Text" = ', data_meduza['text'].isnull().sum())

Кол-во строк с значением NaN в столбце "Text" =  0


Посмотрим сколько элементов Nan в столбце авторов

In [166]:
print('Кол-во строк с значением NaN в столбце "authors" = ', data_meduza['authors'].isnull().sum())

Кол-во строк с значением NaN в столбце "authors" =  106644


Как видим все значения авторов Nan. Можно не использовать данные столбец

Посмотрим, что в столбцах, которые содержат дефис, как обозначение пустых данных

In [17]:
print('edition: ',data_meduza['edition'].unique())
print('reposts_fb: ', data_meduza['reposts_fb'].unique())
print('reposts_vk: ',data_meduza['reposts_vk'].unique())
print('reposts_ok: ',data_meduza['reposts_ok'].unique())
print('reposts_twi: ',data_meduza['reposts_twi'].unique())
print('reposts_lj: ',data_meduza['reposts_lj'].unique())
print('reposts_tg: ',data_meduza['reposts_tg'].unique())
print('likes: ',data_meduza['likes'].unique())
print('views: ',data_meduza['views'].unique())
print('comm_count: ',data_meduza['comm_count'].unique())

edition:  ['-']
reposts_fb:  ['-']
reposts_vk:  ['-']
reposts_ok:  ['-']
reposts_twi:  ['-']
reposts_lj:  ['-']
reposts_tg:  ['-']
likes:  ['-']
views:  ['-']
comm_count:  ['-']


Значений нет, можно удалять

### Рубрики Meduza

In [18]:
# Считаем количество рубрик в данном датасете, а также выведем значения
print('Количество рубрик = ', data_meduza.topics.unique().shape)
data_meduza['topics'].unique()

Количество рубрик =  (1,)


array([nan])

Все рубрики, с количеством новостей в каждой группе

In [19]:
data_meduza.groupby('topics')['topics'].count()

Series([], Name: topics, dtype: int64)

*Вывод:* Рубрик в данном датасете нет